In [1]:
import numpy as np
import pandas as pd

In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset=pd.get_dummies(dataset,drop_first=True)

In [5]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [6]:
dataset=dataset.drop("User ID",axis=1)

In [7]:
dataset["Purchased"].value_counts()

0    257
1    143
Name: Purchased, dtype: int64

In [8]:
indep=dataset[["Age","EstimatedSalary","Gender_Male"]]
dep=dataset["Purchased"]

In [9]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(indep, dep, test_size = 1/3, random_state = 0)

In [10]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
param_grid = {'criterion':['gini', 'entropy', 'log_loss'],
             'max_features':['auto','sqrt','log2'],
             'n_estimators':[50,100]}
grid = GridSearchCV(RandomForestClassifier(),param_grid,refit=True,verbose=3,n_jobs=-1,scoring='f1_weighted')
grid.fit(x_train,y_train)     

Fitting 5 folds for each of 18 candidates, totalling 90 fits


C:\Users\Genuine\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
30 fits failed out of a total of 90.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Genuine\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Genuine\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py", line 458, in fit
    for i, t in enumerate(trees)
  File "C:\Users\Genuine\Anaconda3\lib\site-packages\joblib\parallel.py", line 921, in __call__
    if self.dispatch_one_batch(iterator):
  File "C:\Users\Genuine\Anaconda3\

GridSearchCV(estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'n_estimators': [50, 100]},
             scoring='f1_weighted', verbose=3)

In [12]:
re=grid.cv_results_
grid_predictions = grid.predict(x_test) 
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)   
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)

In [13]:
 from sklearn.metrics import f1_score
f1_macro = f1_score(y_test, grid_predictions, average = 'weighted')
print('The f1_macro value for best parameter {}'.format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'criterion': 'entropy', 'max_features': 'sqrt', 'n_estimators': 100} 0.918082172961622


In [14]:
print('The Confusion Matrix:\n',cm)

The Confusion Matrix:
 [[79  6]
 [ 5 44]]


In [15]:
print('The Report:\n',clf_report)

The Report:
               precision    recall  f1-score   support

           0       0.94      0.93      0.93        85
           1       0.88      0.90      0.89        49

    accuracy                           0.92       134
   macro avg       0.91      0.91      0.91       134
weighted avg       0.92      0.92      0.92       134



In [16]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(x_test)[:,1])

0.9653061224489796

In [17]:
table = pd.DataFrame.from_dict(re)

In [18]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.149463,0.019054,0.015627,8.341244e-07,gini,auto,50,"{'criterion': 'gini', 'max_features': 'auto', ...",0.847141,0.868752,0.851527,0.925272,0.943041,0.887147,0.039459,12
1,0.257756,0.006991,0.028128,6.249905e-03,gini,auto,100,"{'criterion': 'gini', 'max_features': 'auto', ...",0.867478,0.889022,0.851527,0.925272,0.943699,0.895400,0.034555,5
2,0.134386,0.007657,0.018751,6.251693e-03,gini,sqrt,50,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.826263,0.888107,0.851527,0.944161,0.943699,0.890751,0.047666,10
3,0.250018,0.009881,0.033152,1.057542e-02,gini,sqrt,100,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.847141,0.888107,0.851527,0.944161,0.962264,0.898640,0.047123,4
4,0.139630,0.008386,0.015627,1.314549e-06,gini,log2,50,"{'criterion': 'gini', 'max_features': 'log2', ...",0.847141,0.888107,0.851527,0.925272,0.943699,0.891149,0.038579,8
5,0.253141,0.006253,0.028128,6.250930e-03,gini,log2,100,"{'criterion': 'gini', 'max_features': 'log2', ...",0.826263,0.888107,0.851527,0.925272,0.962264,0.890687,0.049010,11
6,0.128134,0.006250,0.015628,1.800652e-06,entropy,auto,50,"{'criterion': 'entropy', 'max_features': 'auto...",0.867478,0.888107,0.851527,0.925272,0.943699,0.895217,0.034591,6
7,0.256267,0.015933,0.025003,7.654812e-03,entropy,auto,100,"{'criterion': 'entropy', 'max_features': 'auto...",0.867478,0.888107,0.851527,0.925272,0.962264,0.898930,0.040146,2
8,0.131261,0.012501,0.015625,2.105656e-06,entropy,sqrt,50,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.867478,0.888107,0.851527,0.925272,0.943041,0.895085,0.034407,7
9,0.265641,0.009883,0.031251,3.033074e-06,entropy,sqrt,100,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.867478,0.907177,0.851527,0.925272,0.962264,0.902744,0.039841,1


In [19]:
age_input=int(input("Age:"))
estimatedsalary_input=float(input("EstimatedSalary:"))
gender_male_input=int(input("Gender_Male 0 or 1:"))

Age:19
EstimatedSalary:19000
Gender_Male 0 or 1:1


In [20]:
Future_Prediction=grid.predict([[age_input,estimatedsalary_input,gender_male_input]])
print('Future_Prediction = {}'.format(Future_Prediction))

Future_Prediction = [1]
